In [20]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
from datetime import date, timedelta

# --- CONNEXION À LA BASE DE DONNÉES ---
db_user = 'root'
db_password = 'root' # Adapte avec ton vrai mot de passe
db_host = 'localhost'
db_port = '3306'
db_name = 'sicda_easytime'

# Crée le "moteur" de connexion
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

print("Connexion à la base de données prête.")

Connexion à la base de données prête.


In [21]:
print("--- Étape 1: Chargement des données brutes ---")
df_absences = pd.read_sql("SELECT USER_FK as employe_id, DATE_DEBUT as date_debut, DATE_REPRISE as date_fin FROM absence WHERE STATUT = 'workflow_status_validated'", engine)
df_conges = pd.read_sql("SELECT USER_FK as employe_id, DATE_DEBUT as date_debut, DATE_REPRISE as date_fin FROM conge WHERE STATUT = 'workflow_status_validated'", engine)
df_employes = pd.read_sql("SELECT ID as employe_id, DATE_EMB as date_embauche, PROFIL_METIER_FK as profil_metier_id, NOEUD_FK as noeud_id FROM utilisateur", engine)
df_feries = pd.read_sql("SELECT DATE_DEBUT as date FROM jr_ferie", engine)

print("Données chargées.")

--- Étape 1: Chargement des données brutes ---
Données chargées.


In [22]:
print("--- Étape 2: Préparation et nettoyage ---")

# Nettoyage des dates
df_absences['date_debut'] = pd.to_datetime(df_absences['date_debut']).dt.date
df_absences['date_fin'] = pd.to_datetime(df_absences['date_fin']).dt.date
df_conges['date_debut'] = pd.to_datetime(df_conges['date_debut']).dt.date
df_conges['date_fin'] = pd.to_datetime(df_conges['date_fin']).dt.date
df_feries['date'] = pd.to_datetime(df_feries['date']).dt.date
df_employes['date_embauche'] = pd.to_datetime(df_employes['date_embauche']).dt.date

# Créer un set de jours fériés pour une recherche rapide
feries_set = set(df_feries['date'])

# Créer un dictionnaire des absences pour une recherche rapide
absences_dict = {}
all_absences = pd.concat([df_absences, df_conges])
for index, row in all_absences.iterrows():
    if row['employe_id'] not in absences_dict:
        absences_dict[row['employe_id']] = []
    absences_dict[row['employe_id']].append((row['date_debut'], row['date_fin']))

print("Données préparées. Affichage d'un aperçu :")
all_absences.head()

--- Étape 2: Préparation et nettoyage ---
Données préparées. Affichage d'un aperçu :


,employe_id,date_debut,date_fin
0,464,2023-04-25,2023-04-26
1,464,2023-04-27,2023-04-27
2,464,2023-04-28,2023-04-28
0,464,2023-04-24,2023-04-25
1,464,2023-04-27,2023-04-30


In [23]:
print("--- Étape 3: Génération du dataset jour par jour ---")
start_date = date.today() - timedelta(days=365*2)
end_date = date.today()
date_range = pd.date_range(start_date, end_date)
dataset_rows = []

for _, employe in df_employes.iterrows():
    for single_date_dt in date_range:
        single_date = single_date_dt.date()
        if employe['date_embauche'] and single_date < employe['date_embauche']:
            continue

        jour_semaine = single_date.weekday()
        jour_mois = single_date.day
        mois = single_date.month
        semaine_annee = single_date.isocalendar()[1]
        est_ferie = 1 if single_date in feries_set else 0
        veille_ferie = 1 if (single_date + timedelta(days=1)) in feries_set else 0
        lendemain_ferie = 1 if (single_date - timedelta(days=1)) in feries_set else 0
        
        est_absent = 0
        if employe['employe_id'] in absences_dict:
            for debut, fin in absences_dict[employe['employe_id']]:
                if debut <= single_date <= fin:
                    est_absent = 1
                    break
        
        dataset_rows.append({
            'employe_id': employe['employe_id'], 'date': single_date,
            'jour_semaine': jour_semaine, 'jour_mois': jour_mois, 'mois': mois,
            'semaine_annee': semaine_annee, 'profil_metier_id': employe['profil_metier_id'],
            'noeud_id': employe['noeud_id'], 'est_ferie': est_ferie,
            'veille_ferie': veille_ferie, 'lendemain_ferie': lendemain_ferie,
            'est_absent': est_absent
        })

print(f"Dataset brut généré avec {len(dataset_rows)} lignes.")

--- Étape 3: Génération du dataset jour par jour ---
Dataset brut généré avec 9444 lignes.


In [24]:
# ==============================================================================
# CELLULE 4.5 : AUGMENTATION DE DONNÉES (VERSION PLUS AGRESSIVE)
# ==============================================================================
augmented_rows = list(dataset_rows)
added_absences_count = 0

# Règle 1 : Lendemain de jours fériés -> On augmente la probabilité
for i in range(len(augmented_rows)):
    row = augmented_rows[i]
    if row['lendemain_ferie'] == 1 and row['est_absent'] == 0:
        # ANCIEN: 0.20 -> NOUVEAU: 0.40 (40% de chance)
        if np.random.rand() < 0.40:
            augmented_rows[i]['est_absent'] = 1
            added_absences_count += 1

# Règle 2 : Vendredis du mois d'août -> On augmente la probabilité
for i in range(len(augmented_rows)):
    row = augmented_rows[i]
    if row['jour_semaine'] == 4 and row['mois'] == 8 and row['est_absent'] == 0:
        # ANCIEN: 0.15 -> NOUVEAU: 0.35 (35% de chance)
        if np.random.rand() < 0.35:
            augmented_rows[i]['est_absent'] = 1
            added_absences_count += 1

# Règle 3 : Un employé spécifique le lundi -> On augmente la probabilité
EMPLOYE_CIBLE_ID = 464
for i in range(len(augmented_rows)):
    row = augmented_rows[i]
    if row['employe_id'] == EMPLOYE_CIBLE_ID and row['jour_semaine'] == 0 and row['est_absent'] == 0:
         # ANCIEN: 0.25 -> NOUVEAU: 0.50 (50% de chance)
         if np.random.rand() < 0.50:
            augmented_rows[i]['est_absent'] = 1
            added_absences_count += 1
            
print(f"{added_absences_count} absences synthétiques ont été ajoutées.")
dataset_rows = augmented_rows

95 absences synthétiques ont été ajoutées.


In [25]:
print("--- Étape 4: Création du DataFrame final et sauvegarde ---")
final_dataset = pd.DataFrame(dataset_rows)
final_dataset = final_dataset[final_dataset['jour_semaine'] < 5] # On supprime les week-ends

output_path = 'data/dataset_absences.csv' # On le sauvegarde à la racine du projet IA
final_dataset.to_csv(output_path, index=False)

print(f"Dataset final sauvegardé dans '{output_path}' avec {len(final_dataset)} lignes.")
print("\n--- Aperçu du dataset final ---")
final_dataset.head()

--- Étape 4: Création du DataFrame final et sauvegarde ---
Dataset final sauvegardé dans 'data/dataset_absences.csv' avec 6734 lignes.

--- Aperçu du dataset final ---


,employe_id,date,jour_semaine,jour_mois,mois,semaine_annee,profil_metier_id,noeud_id,est_ferie,veille_ferie,lendemain_ferie,est_absent
2,1,2023-07-24,0,24,7,30,67,27,0,0,0,0
3,1,2023-07-25,1,25,7,30,67,27,0,0,0,0
4,1,2023-07-26,2,26,7,30,67,27,0,0,0,0
5,1,2023-07-27,3,27,7,30,67,27,0,0,0,0
6,1,2023-07-28,4,28,7,30,67,27,0,0,0,0


In [26]:
print("--- Analyse de la Cible ---")
print(final_dataset['est_absent'].value_counts(normalize=True))

--- Analyse de la Cible ---
est_absent
0    0.984704
1    0.015296
Name: proportion, dtype: float64
